# Beten nach Daten: statistiche Übersichten betr. Datum, Format und Entstehungsort

<a href="https://colab.research.google.com/github/oriflamms/LivreQuanti2023/blob/main/DataModelling/Zeit-Raum-Dimensionen-Deutschland.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment

In [ ]:
import os, pandas as pd, numpy as np
from urllib.request import urlopen

from scipy.stats import chi2_contingency
print(os.getcwd())
# Set the maximum number of rows and columns to display
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Data (capta)

## Load data and check contents

Only if on Google Colab. Skip if used locally

In [ ]:
file_url_1 = 'https://raw.githubusercontent.com/oriflamms/LivreQuanti2023/main/DataModelling/Export_stutzmann_horae_t53_Manuscript_rich_20231109.xlsx'
file_url_2 = 'https://raw.githubusercontent.com/oriflamms/LivreQuanti2023/main/DataModelling/t53_Manuscript_Country.csv'
!wget {file_url_1}
!wget {file_url_2}

Back to main track if repo is cloned and used locally

In [ ]:
#df = pd.read_excel('./Export_stutzmann_horae_t53_numformat.xlsx', sheet_name='Capta')
df = pd.read_excel('./Export_stutzmann_horae_t53_Manuscript_rich_20231109.xlsx', sheet_name='Tabelle2')
df_country = pd.read_csv('./t53_Manuscript_Country.csv', sep=';')

In [ ]:
print(df.shape)
print(list(df.columns))
df.head()

In [ ]:
print(df['leaf dimensions : height (mm)'].dtypes)
# print(df['leaf dimensions : height (mm)'].unique())
print(df['origDate'].dtypes)
# print(df['origDate'].unique())
print(df['origDate : from'].dtypes)
# print(df['origDate : from'].unique())
print(df['origDate : to'].dtypes)
# print(df['origDate : to'].unique())


In [ ]:
df_country.head()

In [ ]:
# Merge dataframes based on "Manuscript H-ID"
df = pd.merge(df, df_country[['Manuscript H-ID', 'Country']], on='Manuscript H-ID', how='left')

df.shape

In [ ]:
df.head()

In [ ]:
df['leaf dimensions : height (mm)'] = pd.to_numeric(df['leaf dimensions : height (mm)'], errors='coerce')
df['origDate'] = pd.to_numeric(df['origDate'], errors='coerce')
df['origDate : from'] = pd.to_numeric(df['origDate : from'], errors='coerce')
df['origDate : to'] = pd.to_numeric(df['origDate : to'], errors='coerce')
df['Country'] = df['Country'].astype(str)

In [ ]:
print(df['leaf dimensions : height (mm)'].dtypes)
print(sorted(df['leaf dimensions : height (mm)'].unique()))
# print(df['origDate'].dtypes)
print(sorted(df['origDate'].unique()))
print(df['origDate : from'].dtypes)
print(sorted(df['origDate : from'].unique()))
print(df['origDate : to'].dtypes)
print(sorted(df['origDate : to'].unique()))
print(df['Country'].dtypes)
print(sorted(df['Country'].unique()))


In [ ]:
# Define a function to calculate the "origDate-avg"
def calculate_origDate_avg(row):
    if not pd.isna(row['origDate']):
        return row['origDate']
    elif not pd.isna(row['origDate : from']) and not pd.isna(row['origDate : to']):
        return (row['origDate : from'] + row['origDate : to']) / 2
    else:
        return np.nan

# Apply the function to create the new column
df['origDate-avg'] = df.apply(calculate_origDate_avg, axis=1)
# df['origDate-avg'] = df['origDate-avg'].round().astype(int)

In [ ]:
df.head()

Convert data to numeric if needed. Reduce scope.

In [ ]:
# Drop rows with NaN, 0, or blank values in columns A and C
df = df.dropna(subset=['leaf dimensions : height (mm)'])
df = df[df['leaf dimensions : height (mm)'] != 0]

df.shape

In [ ]:
def process_country(value):
    countries = [country.strip() for country in value.split('|') if country.strip()]
    
    # Check if all values are the same
    if len(set(countries)) == 1:
        return countries[0]
    else:
        # If multiple values, return sorted and unique values
        # return '|'.join(sorted(set(countries)))
        return 'multiple countries'

# Apply the function to create the new "Processed_Country" column
df['Processed_Country'] = df['Country'].apply(process_country)

# Categories and pivot tables

## Dimensions / Country

In [ ]:
# Assuming 'df' is your DataFrame
pivot_table_avg_dimensions_by_Country = df.pivot_table(values='leaf dimensions : height (mm)', index='Processed_Country', aggfunc='mean').round(1)

# Display the pivot table
print(pivot_table_avg_dimensions_by_Country)


## Dimensions by DateRange (span of 50 years)

In [ ]:
# Convert 'origDate-avg' to numeric, coercing errors to NaN
df['origDate-avg'] = pd.to_numeric(df['origDate-avg'], errors='coerce')

# Create a new column for the date range categories
bins = range(1001, 2001, 50)
labels = [f"{start}-{start+49}" for start in bins[:-1]]
df['DateRange'] = pd.cut(df['origDate-avg'], bins=bins, labels=labels, right=False)



In [ ]:
pivot_table_avg_dimensions_by_DateRange = df.pivot_table(values='leaf dimensions : height (mm)', index='DateRange', aggfunc='mean').round(1)

# Display the pivot table
print(pivot_table_avg_dimensions_by_DateRange)


In [ ]:
# Pivot table for average dimensions by DateRange and Country
pivot_table_avg_dimensions = df.pivot_table(
    values='leaf dimensions : height (mm)',
    index=['DateRange', 'Processed_Country'],
    aggfunc='mean',
    margins=True,  # Add subtotals
    margins_name='missing',  # Specify the name for the subtotal row
).round(1)

In [ ]:
pivot_table_avg_dimensions

In [ ]:
# Pivot table for average dimensions by DateRange and Country
pivot_table_avg_dimensions_full = df.pivot_table(
    values=['leaf dimensions : height (mm)', 'Manuscript H-ID'],
    index=['DateRange', 'Processed_Country'],
    aggfunc={'leaf dimensions : height (mm)': 'mean', 'Manuscript H-ID': 'count'},
    margins=True,  # Add subtotals
    margins_name='missing',  # Specify the name for the subtotal row
).round(1)

pivot_table_avg_dimensions_full.head()


# Correlation (khi2)

## Date and Country

In [ ]:
# Create a contingency table
contingency_table = pd.crosstab(df['Processed_Country'], df['DateRange'])

# Perform the chi-squared test
chi2, p, _, _ = chi2_contingency(contingency_table)

# Print the results
print(f"Chi-squared value: {chi2}")
print(f"P-value: {p}")

## Size and Date

In [ ]:
# Create a new column for the date range categories
bins = range(0, 900, 20)
labels = [f"{start}-{start+19}" for start in bins[:-1]]
df['heightRange'] = pd.cut(df['leaf dimensions : height (mm)'], bins=bins, labels=labels, right=False)


# Create a contingency table
contingency_table = pd.crosstab(df['heightRange'], df['DateRange'])

# Perform the chi-squared test
chi2, p, _, _ = chi2_contingency(contingency_table)

# Print the results
print(f"Chi-squared value: {chi2}")
print(f"P-value: {p}")

## Country and Size

In [ ]:
# Create a contingency table
contingency_table = pd.crosstab(df['heightRange'], df['Country'])

# Perform the chi-squared test
chi2, p, _, _ = chi2_contingency(contingency_table)

# Print the results
print(f"Chi-squared value: {chi2}")
print(f"P-value: {p}")